# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
import torch
import pickle
import shutil
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from torchmetrics.functional.regression import mean_absolute_error, pearson_corrcoef
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.utils.hash import dict_hash
from src.pt.hyper_opt import train_hyper_opt
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
from regression_bias_corrector import LinearBiasCorrector
import optuna
from sklearn.preprocessing import LabelEncoder
from plottable import ColumnDefinition, Table
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import matplotlib.lines as mlines
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats
from functools import reduce
import shutil
import os
from pypdf import PdfReader
import re


def make_rgb_transparent(rgb, bg_rgb, alpha):
    return [alpha * c1 + (1 - alpha) * c2 for (c1, c2) in zip(rgb, bg_rgb)]

# Check initial data files

In [ ]:
path = 'E:/YandexDisk/Work/bbd/millennium'

data_anl = pd.read_excel(f"{path}/Результаты анализов/data.xlsx", index_col=0)
data_anl['Дата обследования'] = data_anl['Дата обследования'].dt.date
data_anl.insert(0, 'ID', data_anl['ФИО'].astype(str) + ' ' + data_anl['Дата обследования'].astype(str))
ids_counts_anl = data_anl['ФИО'].value_counts()
sorter_anl = ids_counts_anl.index.to_list()
data_anl.sort_values(by="ФИО", key=lambda column: column.map(lambda e: sorter_anl.index(e)), inplace=True)
feats_anl = pd.read_excel(f"{path}/Результаты анализов/feats_with_metrics.xlsx", index_col=0)

data_ecg = pd.read_excel(f"{path}/Результаты экг/data.xlsx", index_col=0)
data_ecg['Дата обследования'] = data_ecg['Дата и время обследования'].dt.date
data_ecg.insert(0, 'ID', data_ecg['ФИО'].astype(str) + ' ' + data_ecg['Дата обследования'].astype(str))
ids_counts_ecg = data_ecg['ФИО'].value_counts()
sorter_ecg = ids_counts_ecg.index.to_list()
data_ecg.sort_values(by="ФИО", key=lambda column: column.map(lambda e: sorter_ecg.index(e)), inplace=True)
feats_ecg = pd.read_excel(f"{path}/Результаты экг/feats_with_metrics.xlsx", index_col=0)

data_bcp = pd.read_excel(f"{path}/Результаты медасс/data.xlsx", index_col=0)
data_bcp['Дата обследования'] = data_bcp['Дата и время обследования'].dt.date
data_bcp.insert(0, 'ID', data_bcp['ФИО'].astype(str) + ' ' + data_bcp['Дата обследования'].astype(str))
ids_counts_bcp = data_bcp['ФИО'].value_counts()
sorter_bcp = ids_counts_bcp.index.to_list()
data_bcp.sort_values(by="ФИО", key=lambda column: column.map(lambda e: sorter_bcp.index(e)), inplace=True)
feats_bcp = pd.read_excel(f"{path}/Результаты медасс/feats_with_metrics.xlsx", index_col=0)

In [ ]:
print(data_anl.index.is_unique)
print(data_ecg.index.is_unique)
print(data_bcp.index.is_unique)

In [ ]:
print(len(set.intersection(set(data_anl['ФИО']), set(data_ecg['ФИО']))))
print(len(set.intersection(set(data_bcp['ФИО']), set(data_ecg['ФИО']))))
print(len(set.intersection(set(data_anl['ФИО']), set(data_bcp['ФИО']))))
print(len(set.intersection(set(data_anl['ФИО']), set(data_bcp['ФИО']), set(data_ecg['ФИО']))))

In [ ]:
print(len(set.intersection(set(data_anl['ID']), set(data_ecg['ID']))))
print(len(set.intersection(set(data_bcp['ID']), set(data_ecg['ID']))))
print(len(set.intersection(set(data_anl['ID']), set(data_bcp['ID']))))
print(len(set.intersection(set(data_anl['ID']), set(data_bcp['ID']), set(data_ecg['ID']))))

In [ ]:
data_anl_new = data_anl.set_index('ID')
data_anl_new.loc[data_anl_new.index.duplicated(keep=False), :]

In [ ]:
data_ecg_new = data_ecg.set_index('ID')
data_ecg_new.loc[data_ecg_new.index.duplicated(keep=False), :]

In [ ]:
data_bcp_new = data_bcp.set_index('ID')
data_bcp_new.loc[data_bcp_new.index.duplicated(keep=False), :]

# Load components

In [ ]:
dir_root = f"E:/Git/MillenniumAge"

feat_trgt = 'Возраст'

components = {
    'Оценка состава тела, женщины': {
        'name': 'Оценка состава тела',
        'path': f"{dir_root}/data/Оценка состава тела/females",
        'bkg_count': 128,
        'likelihood': 0.5
    },
    'Оценка состава тела, мужчины': {
        'name': 'Оценка состава тела',
        'path': f"{dir_root}/data/Оценка состава тела/males",
        'bkg_count': 50,
        'likelihood': 0.8
    },
    
    'Электрокардиограмма, все': {
        'name': 'Электрокардиограмма',
        'path': f"{dir_root}/data/Электрокардиограмма/all",
        'bkg_count': 200,
        'likelihood': 0.8
    },
    'Электрокардиограмма, старше 15': {
        'name': 'Электрокардиограмма',
        'path': f"{dir_root}/data/Электрокардиограмма/over15",
        'bkg_count': 180,
        'likelihood': 0.66
    },
    'Электрокардиограмма, младше 15': {
        'name': 'Электрокардиограмма',
        'path': f"{dir_root}/data/Электрокардиограмма/under15",
        'bkg_count': 30,
        'likelihood': 0.66
    },
    
    'Гематологические исследования, все': {
        'name': 'Гематологические исследования',
        'path': f"{dir_root}/data/Гематологические исследования/all",
        'bkg_count': 600,
        'likelihood': 0.6
    },
    'Гематологические исследования, старше 15': {
        'name': 'Гематологические исследования',
        'path': f"{dir_root}/data/Гематологические исследования/over15",
        'bkg_count': 520,
        'likelihood': 0.4
    },
    'Гематологические исследования, младше 15': {
        'name': 'Гематологические исследования',
        'path': f"{dir_root}/data/Гематологические исследования/under15",
        'bkg_count': 80,
        'likelihood': 0.6
    },
    
    'Биохимические исследования 7, все': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/7/all",
        'bkg_count': 280,
        'likelihood': 0.77
    },
    'Биохимические исследования 7, старше 15': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/7/over15",
        'bkg_count': 250,
        'likelihood': 0.6
    },
    'Биохимические исследования 7, младше 15': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/7/under15",
        'bkg_count': 36,
        'likelihood': 0.9
    },
    'Биохимические исследования 9, все': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/9/all",
        'bkg_count': 250,
        'likelihood': 0.8
    },
    'Биохимические исследования 9, старше 15': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/9/over15",
        'bkg_count': 200,
        'likelihood': 0.62
    },
    'Биохимические исследования 9, младше 15': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/9/under15",
        'bkg_count': 32,
        'likelihood': 0.9
    },
    'Биохимические исследования 12': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/12",
        'bkg_count': 180,
        'likelihood': 0.65
    },
    'Биохимические исследования 23': {
        'name': 'Биохимические исследования',
        'path': f"{dir_root}/data/Биохимические исследования/23",
        'bkg_count': 144,
        'likelihood': 0.7
    },
    
    'Половые гормоны, женщины': {
        'name': 'Половые гормоны',
        'path': f"{dir_root}/data/Половые гормоны/females",
        'bkg_count': 62,
        'likelihood': 0.85
    },
    'Половые гормоны, мужчины': {
        'name': 'Половые гормоны',
        'path': f"{dir_root}/data/Половые гормоны/males",
        'bkg_count': 36,
        'likelihood': 0.55
    },
    
    'Гормоны 3, все': {
        'name': 'Гормоны',
        'path': f"{dir_root}/data/Гормоны/3/all",
        'bkg_count': 300,
        'likelihood': 0.4
    },
    'Гормоны 3, старше 18': {
        'name': 'Гормоны',
        'path': f"{dir_root}/data/Гормоны/3/over18",
        'bkg_count': 280,
        'likelihood': 0.17
    },
    'Гормоны 3, младше 18': {
        'name': 'Гормоны',
        'path': f"{dir_root}/data/Гормоны/3/under18",
        'bkg_count': 32,
        'likelihood': 0.5
    },
    'Гормоны 5': {
        'name': 'Гормоны',
        'path': f"{dir_root}/data/Гормоны/5",
        'bkg_count': 200,
        'likelihood': 0.37
    },
    'Гормоны 6': {
        'name': 'Гормоны',
        'path': f"{dir_root}/data/Гормоны/6",
        'bkg_count': 180,
        'likelihood': 0.35
    },
}

feats_all = []
for comp in components:
    components[comp]['data'] = pd.read_excel(f"{components[comp]['path']}/data.xlsx", index_col=0)
    if 'Дата обследования' in components[comp]['data'].columns:
        components[comp]['data']['Дата обследования'] = components[comp]['data']['Дата обследования'].dt.date
    if 'Дата и время обследования' in components[comp]['data'].columns:
        components[comp]['data']['Дата обследования'] = components[comp]['data']['Дата и время обследования'].dt.date
    components[comp]['data'].insert(0, 'ID', components[comp]['data']['ФИО'].astype(str) + ' ' + components[comp]['data']['Дата обследования'].astype(str))
    components[comp]['data'].insert(0, 'index', components[comp]['data'].index.astype(str))
    components[comp]['feats'] = pd.read_excel(f"{components[comp]['path']}/feats.xlsx", index_col=0)
    components[comp]['results'] = pd.read_excel(f"{components[comp]['path']}/model/df.xlsx", index_col=0)
    components[comp]['metrics'] = pd.read_excel(f"{components[comp]['path']}/model/metrics.xlsx", index_col=0)
    components[comp]['model'] = TabularModel.load_model(f"{components[comp]['path']}/model")
    components[comp]['corrector'] = LinearBiasCorrector()
    comp_results = components[comp]['results']
    components[comp]['corrector'].fit(comp_results.loc[comp_results['Group'] == 'Train', feat_trgt].values, comp_results.loc[comp_results['Group'] == 'Train', 'Prediction'].values)
    res_cols = ['Group', 'Prediction', 'Error', 'Prediction Unbiased', 'Error Unbiased']
    components[comp]['data'].loc[components[comp]['data'].index, res_cols] = comp_results.loc[components[comp]['data'].index, res_cols]
    components[comp]['data_shap'] = components[comp]['data'].copy()
    
    feats = components[comp]['feats'].index.values
    feats = feats[feats != feat_trgt]
    feats_all += list(feats)
    feats_all += [f"Предсказание {components[comp]['name']}", f"Возрастная Акселерация {components[comp]['name']}"]
    
    components[comp]['feats_corr'] = pd.DataFrame(index=feats, columns=['Correlation'])
    for f in feats:
        components[comp]['feats_corr'].at[f, 'Correlation'], _ = scipy.stats.pearsonr(components[comp]['data'].loc[:, f].values, components[comp]['data'].loc[:, feat_trgt].values)
        
feats_all = list(dict.fromkeys(feats_all))

In [ ]:
for comp in components:
    print(f"{comp}: {components[comp]['data'].shape[0]}")

# Create united data file

In [ ]:
trgt_components = [
    'Гематологические исследования, старше 15',
    # 'Биохимические исследования 7, старше 15',
    # 'Биохимические исследования 9, старше 15',
    # 'Биохимические исследования 12',
    'Биохимические исследования 23',
    'Половые гормоны, женщины',
    'Половые гормоны, мужчины',
    'Гормоны 6',
    
    'Оценка состава тела, женщины',
    'Оценка состава тела, мужчины',
    
    'Электрокардиограмма, старше 15',
]

dfs = {}
for comp in trgt_components:
    feats = ['Возраст', 'Пол'] + components[comp]['feats'].index.to_list()
    df = components[comp]['data'].set_index('ID').loc[:, feats]
    if not df.index.is_unique:
        print(f"{comp} {df.shape} index unique: {df.index.is_unique}, number of duplicates: {len(df.index[df.index.duplicated()])}")
        df = df[~df.index.duplicated(keep='first')]
    else:
        print(f"{comp} {df.shape} index unique: {df.index.is_unique}")
    dfs[comp] = df

index_cmn = reduce(pd.Index.union, [dfs[comp].index for comp in trgt_components]).to_list()
feats_cmn = ['Возраст', 'Пол'] + reduce(pd.Index.union, [components[comp]['feats'].index for comp in trgt_components]).to_list()

data = pd.DataFrame(index=index_cmn, columns=feats_cmn)
for comp in trgt_components:
    data = data.combine_first(dfs[comp])

print(data.shape)
data.to_excel(f"{dir_root}/data/data.xlsx")

# Unpack ZIP

In [ ]:
path = 'E:/YandexDisk/Work/bbd/millennium/Результаты чекап'
checkup_series = 1
path_to = f"{path}/test"

shutil.unpack_archive(f"{path}/{checkup_series}/example.zip", path_to)

folders = [os.path.split(f.path)[1] for f in os.scandir(path_to) if f.is_dir()]

# Parse folders with samples

In [ ]:
path = 'E:/YandexDisk/Work/bbd/millennium/Результаты чекап/2'
folders = [os.path.split(f.path)[1] for f in os.scandir(path) if f.is_dir()]

dir_root = f"E:/Git/MillenniumAge"

# Анализы
feats_anl = pd.read_excel(f"{dir_root}/data/processing/Результаты анализов/features.xlsx")
if feats_anl['prefix'].is_unique:
    feats_anl.set_index('prefix', inplace=True)
else:
    raise ValueError(f"Features' prefixes are not unique!")
feats_anl_same_line = feats_anl.loc[feats_anl['line'] == 0, :]
feats_anl_diff_line = feats_anl.loc[feats_anl['line'] != 0, :]

feats_anl_same_line_dict = dict(zip(feats_anl_same_line.index.values, feats_anl_same_line['feature'].values))
feats_anl_diff_line_dict = dict(zip(feats_anl_diff_line.index.values, feats_anl_diff_line['feature'].values))

with open(f"{dir_root}/data/processing/Результаты анализов/skip_starts.txt") as f:
    skip_starts_anl = tuple(list(set(f.read().splitlines())))

missed_pages_anl = {}
missed_lines_anl = []

df_anl = pd.DataFrame(columns=['Sample ID', 'Pages', 'File', '№ направления', 'ФИО', 'Дата рождения', 'Дата обследования', 'Пол'] + list(feats_anl['feature'].unique()))

# ЭКГ
feats_ecg = pd.read_excel(f"{dir_root}/data/processing/Результаты экг/features.xlsx", index_col=0)
df_ecg = pd.DataFrame(columns=['Sample ID', 'File'] + feats_ecg.index.to_list())

# Медасс
feats_bcp = pd.read_excel(f"{dir_root}/data/processing/Результаты медасс/features.xlsx", index_col=0)
df_bcp = pd.DataFrame(columns=['Sample ID', 'File'] + feats_bcp.index.to_list())


folders_files_types = {}

for folder in folders:
    files = glob(f"{path}/{folder}/*.pdf")
    
    folders_files_types[folder] = {}
    
    for file in files:
        head, fn = os.path.split(file)
        reader = PdfReader(file)
        
        print(f"{folder}: {fn}")
        
        start_page_lines = reader.pages[0].extract_text().splitlines()
        
        if len(start_page_lines) > 50 and (start_page_lines[48] == 'КОМПЬЮТЕРНАЯ ИНТЕРПРЕТАЦИЯ ЭКГ' or start_page_lines[49] == 'КОМПЬЮТЕРНАЯ ИНТЕРПРЕТАЦИЯ ЭКГ'):
            folders_files_types[folder][file] = 'ЭКГ'
            
            pages = {0: reader.pages[0].extract_text().splitlines()}
            
            # Нет роста или веса
            if not any(x in pages[0][15] for x in ['см', 'кг']):
                pages[0] = pages[0][0:15] + [''] + pages[0][15:]
            # Лишняя строчка - есть ФИО врача    
            if (pages[0][17] != 'Дата/время:') and (pages[0][18] == 'Дата/время:'):
                pages[0].pop(17)
            # Нет номера 
            if pages[0][16] == 'Дата/время:':
                pages[0] = pages[0][0:16] + [''] + pages[0][16:]
                
            sample_name = re.sub(' +', ' ', pages[int(feats_ecg.at['ФИО', 'page'])][int(feats_ecg.at['ФИО', 'row'])]).rstrip()
            sample_number = re.sub(' +', ' ', pages[int(feats_ecg.at['Номер', 'page'])][int(feats_ecg.at['Номер', 'row'])]).rstrip()
            sample_datetime = re.sub(' +', ' ', pages[int(feats_ecg.at['Дата и время обследования', 'page'])][int(feats_ecg.at['Дата и время обследования', 'row'])]).rstrip()
            
            sample_id = folder
            sample_id_second = f"{sample_name} {sample_number} {sample_datetime}"
            
            df_ecg.at[sample_id, 'Sample ID'] = sample_id_second
            df_ecg.at[sample_id, 'File'] = fn
            
            for f in feats_ecg.index:
                
                f_str = re.sub(' +', ' ', pages[int(feats_ecg.at[f, 'page'])][int(feats_ecg.at[f, 'row'])]).rstrip()
                f_re_str = feats_ecg.at[f, 're_string']
                if not pd.isna(f_re_str):
                    f_re_res = re.findall(fr"{f_re_str}", f_str)
                    if len(f_re_res) > 0:
                        f_re_res = f_re_res[0]
                        f_re_group = feats_ecg.at[f, 're_group']
                        if not pd.isna(f_re_group):
                            f_val = f_re_res[int(f_re_group)]
                        else:
                            f_val = f_re_res
                    else:
                        f_val = ''
                else:
                    f_val = f_str
                
                df_ecg.at[sample_id, f] = f_val
            
        elif start_page_lines[2] == 'Оценка состава тела (биоимпедансный анализ)':
            folders_files_types[folder][file] = 'Биоимпеданс'
            
            pages = {}
            for p_id, p in enumerate(reader.pages):
                pages[p_id] = p.extract_text().splitlines()
                
            if len(pages) == 5:
            
                if 'Удельный основной обмен (ккал/м2/сут.)' in pages[0]:
                    pages[0].remove('Удельный основной обмен (ккал/м2/сут.)')
                if 'Удельный основной обмен (ккал/м 2/сут.)' in pages[0]:
                    pages[0].remove('Удельный основной обмен (ккал/м 2/сут.)')
                if 'по БЖМ' in pages[0]:
                    pages[0].remove('по БЖМ')
                if pages[2][7].startswith('Ваш удельный основной обмен:'):
                    pages[2].pop(7)
                skip_rows_page_4 = [
                    'Индекс',
                    'скелетно-мышечной',
                    'массы (кг/м2)',
                    'массы (кг/м 2)',
                    '(кг)'
                ]
                for sr in skip_rows_page_4:
                    if sr in pages[4]:
                        pages[4].remove(sr)
                
                sample_name = re.sub(' +', ' ', pages[int(feats_bcp.at['ФИО', 'page'])][int(feats_bcp.at['ФИО', 'row'])]).rstrip()
                datetime_re_str = feats_bcp.at['Дата и время обследования', 're_string']
                sample_datetime = re.findall(fr"{datetime_re_str}", pages[int(feats_bcp.at['Дата и время обследования', 'page'])][int(feats_bcp.at['Дата и время обследования', 'row'])])[0]
                
                sample_id = folder
                sample_id_second = f"{sample_name} {sample_datetime}"
                
                df_bcp.at[sample_id, 'File'] = fn
                df_bcp.at[sample_id, 'Sample ID'] = sample_id_second
                
                for f in feats_bcp.index:
                    
                    f_str = re.sub(' +', ' ', pages[int(feats_bcp.at[f, 'page'])][int(feats_bcp.at[f, 'row'])]).rstrip()
                    f_re_str = feats_bcp.at[f, 're_string']
                    if not pd.isna(f_re_str):
                        f_re_res = re.findall(fr"{f_re_str}", f_str)[0]
                        f_re_group = feats_bcp.at[f, 're_group']
                        if not pd.isna(f_re_group):
                            f_val = f_re_res[int(f_re_group)]
                        else:
                            f_val = f_re_res
                    else:
                        f_val = f_str
                    
                    df_bcp.at[sample_id, f] = f_val
            
        else:
            folders_files_types[folder][file] = 'Другое'
            
            pages = {}
            for p_id, p in enumerate(reader.pages):
                lines = p.extract_text().splitlines()
                
                if lines[0] == 'Фамилия:':
                    
                    line_sex = 7
                    line_birth = 4
                    if lines[1] == 'Дата рождения:ЛПУ:':
                        line_sex = 6
                        line_birth = 3
                    
                    if lines[10].startswith('Дата:'):
                        line_date = 10
                        line_name = 12
                    elif lines[9].startswith('Дата:'):
                        line_date = 9
                        line_name = 11
                    else:
                        raise ValueError(f"Wrong ID parsing: {fn} {p_id}")
                        
                    sample_name = lines[line_name].capitalize() + ' ' + re.findall(r"(.*)Имя", lines[line_name - 1])[0]
                    sample_date = re.findall(r"Дата: (.*)", lines[line_date])[0]
                    sample_number = lines[3]
                    
                    sample_id = folder
                    sample_id_second = f"{sample_name} {sample_date} {sample_number}"
                    
                    if sample_id in df_anl.index:
                        df_anl.at[sample_id, 'Pages'] += f' {p_id}'
                    else:
                        df_anl.at[sample_id, 'Pages'] = f'{p_id}'

                    df_anl.at[sample_id, 'File'] = fn
                    df_anl.at[sample_id, 'Sample ID'] = sample_id_second
                    df_anl.at[sample_id, '№ направления'] = sample_number
                    df_anl.at[sample_id, 'ФИО'] = sample_name
                    df_anl.at[sample_id, 'Дата рождения'] = lines[line_birth].replace('17.03.7979', '17.03.1979')
                    df_anl.at[sample_id, 'Дата обследования'] = sample_date
                    df_anl.at[sample_id, 'Пол'] = re.findall(r"Пол: (.+)", lines[line_sex])[0][0]

                    for line_id, line in enumerate(lines):
                        line = line.replace("не обнаружено", "0.0")
                        line = line.replace("\u2009", "")
                        line = line.replace("в 1 мл", "в мл")
                        if line in feats_anl_diff_line_dict:
                            target_line = lines[line_id + feats_anl_diff_line.at[line, 'line']]
                            line_parse = re.findall(fr"([-+]?(?:\d+\.\d+|\d+|\.\d+)).*", target_line)
                            df_anl.at[sample_id, feats_anl_diff_line_dict[line]] = line_parse[0]
                        else:
                            line = line.replace(" - ", "-")
                            line_parse_w_units = re.findall(fr"(.*)\s([-+]?(?:\d+\.\d+|\d+|\.\d+)) (.*)", line)
                            line_parse_wo_units = re.findall(fr"(.*)\s([-+]?(?:\d+\.\d+|\d+|\.\d+)) ", line)
                            if len(line_parse_w_units) > 0:
                                if line_parse_w_units[0][0] in feats_anl_same_line_dict:
                                    feat_unit = feats_anl_same_line.at[line_parse_w_units[0][0], 'unit']
                                    if not pd.isna(feat_unit):
                                        if feat_unit in line_parse_w_units[0][2] or feat_unit.replace('МЕ/', 'Ед/') in line_parse_w_units[0][2]:
                                            df_anl.at[sample_id, feats_anl_same_line_dict[line_parse_w_units[0][0]]] = line_parse_w_units[0][1]
                                        else:
                                            print(f"{line} ({fn} {p_id} {line_id})")
                                    else:
                                        df_anl.at[sample_id, feats_anl_same_line_dict[line_parse_w_units[0][0]]] = line_parse_w_units[0][1]
                            elif len(line_parse_wo_units) > 0:
                                print(f"Check: {line_parse_wo_units}")
                                df_anl.at[sample_id, feats_anl_same_line_dict[line_parse_wo_units[0][0]]] = line_parse_wo_units[0][1]

df_anl = df_anl.apply(pd.to_numeric, errors='ignore')
df_anl['Дата рождения'] = pd.to_datetime(df_anl['Дата рождения'], format="%d.%m.%Y").dt.date
df_anl['Дата обследования'] = pd.to_datetime(df_anl['Дата обследования'], format="%d.%m.%Y").dt.date
df_anl.insert(7, 'Возраст', (df_anl['Дата обследования'] - df_anl['Дата рождения']) / np.timedelta64(1, 'D') / 365.25)
df_anl = df_anl.loc[:, ['Возраст', 'Пол'] + list(feats_anl['feature'].unique())]
df_anl.dropna(axis=1, how='all', inplace=True)

df_ecg = df_ecg.apply(pd.to_numeric, errors='ignore')
df_ecg['Пол'] = df_ecg['Пол'].str.upper()
df_ecg['Дата и время обследования'] = pd.to_datetime(df_ecg['Дата и время обследования'], format="%d.%m.%Y %H:%M:%S")
df_ecg = df_ecg.loc[:, feats_ecg.index.to_list()]
df_ecg.dropna(axis=1, how='all', inplace=True)

df_bcp = df_bcp.apply(pd.to_numeric, errors='ignore')
df_bcp['Дата и время обследования'] = pd.to_datetime(df_bcp['Дата и время обследования'], format="%d.%m.%Y %H:%M:%S")
df_bcp = df_bcp.loc[:, feats_bcp.index.to_list()]
df_bcp.dropna(axis=1, how='all', inplace=True)

index_cmn = reduce(pd.Index.union, [x.index for x in [df_anl, df_bcp, df_ecg]]).to_list()
feats_cmn = df_anl.columns.to_list() + df_ecg.columns.to_list() + df_bcp.columns.to_list()
feats_cmn = list(dict.fromkeys(feats_cmn))

data = pd.DataFrame(index=index_cmn, columns=feats_cmn)
for x in [df_anl, df_bcp, df_ecg]:
    data = data.combine_first(x)

In [ ]:
feats_bcp.at[f, 're_string']

In [ ]:
df_ecg.columns[df_ecg.columns.duplicated()]

In [ ]:
path = 'E:/YandexDisk/Work/bbd/millennium/Результаты чекап/3'
folder = 'Бармина Т. Н' #'Авдеева Г. Н' #'Бармина Т. Н'
file = 'cf16542c2ce505abe7da0de1f8ed430e.pdf' #'Биоимпеданс.pdf' #'cf16542c2ce505abe7da0de1f8ed430e.pdf'
reader = PdfReader(f"{path}/{folder}/{file}")
print(reader.pages[4].extract_text())